<a href="https://colab.research.google.com/github/apester/PINN/blob/main/Advection_diffusion_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cell 0 — Colab setup (optional)

In [ ]:
!nvidia-smi

Cell 1 — Imports + device

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

torch.set_num_threads(1)  # fine on Colab too
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Cell 2 — Problem setup (your parameters)

In [ ]:
L = 2.0
u = 1.0
D = 0.5
T_final = 0.10

N = 1000
dx = L / N

dt_adv  = dx / abs(u)
dt_diff = dx*dx / (2*D)
dt = 0.9 * min(dt_adv, dt_diff)

Nt = int(np.ceil(T_final / dt))
dt = T_final / Nt

lam = u * dt / dx
mu  = D * dt / (dx*dx)

(dx, dt, Nt, lam, mu)

Cell 3 — Ground truth (explicit Euler, upwind + diffusion)

In [ ]:
x = np.linspace(0.0, L, N+1, dtype=np.float32)
t = np.linspace(0.0, T_final, Nt+1, dtype=np.float32)

c = np.empty((Nt+1, N+1), dtype=np.float32)

# IC: c(x,0)=x
c[0, :] = x

# Dirichlet BCs: c(0,t)=0, c(L,t)=L
def g0(tt): return 0.0
def gL(tt): return L

lam32 = np.float32(lam)
mu32  = np.float32(mu)

for n in range(Nt):
    c[n, 0]  = g0(t[n])
    c[n, -1] = gL(t[n])

    cn = c[n, :]
    cn1 = c[n+1, :]

    cn1[1:-1] = (
        cn[1:-1]
        - lam32 * (cn[1:-1] - cn[:-2])
        + mu32  * (cn[2:] - 2.0*cn[1:-1] + cn[:-2])
    )

    cn1[0]  = g0(t[n+1])
    cn1[-1] = gL(t[n+1])

c.min(), c.max(), c.shape

Cell 4 — Ground truth sampler (bilinear interpolation)

In [ ]:
def c_groundtruth(xq, tq):
    xq = np.asarray(xq, dtype=np.float32).reshape(-1)
    tq = np.asarray(tq, dtype=np.float32).reshape(-1)

    ix = np.clip(xq / dx, 0, N).astype(np.float32)
    it = np.clip(tq / dt, 0, Nt).astype(np.float32)

    i0 = np.floor(ix).astype(np.int32)
    i1 = np.clip(i0 + 1, 0, N)
    j0 = np.floor(it).astype(np.int32)
    j1 = np.clip(j0 + 1, 0, Nt)

    wx = ix - i0
    wt = it - j0

    c00 = c[j0, i0]
    c10 = c[j0, i1]
    c01 = c[j1, i0]
    c11 = c[j1, i1]

    cx0 = (1-wx)*c00 + wx*c10
    cx1 = (1-wx)*c01 + wx*c11
    out = (1-wt)*cx0 + wt*cx1
    return out.reshape(-1,1).astype(np.float32)

# sanity check
xx = np.array([[0.1],[1.3],[2.0]], dtype=np.float32)
tt = np.zeros_like(xx)
np.hstack([xx, c_groundtruth(xx, tt)])

Cell 5 — PINN model

In [ ]:
def normalize_xt(x_in, t_in):
    x_n = 2.0*(x_in/L) - 1.0
    t_n = 2.0*(t_in/T_final) - 1.0
    return x_n, t_n

class PINN(nn.Module):
    def __init__(self, width=64, depth=4):
        super().__init__()
        layers = []
        in_dim = 2
        for _ in range(depth):
            layers += [nn.Linear(in_dim, width), nn.Tanh()]
            in_dim = width
        layers += [nn.Linear(in_dim, 1)]
        self.net = nn.Sequential(*layers)

        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x, t):
        x_n, t_n = normalize_xt(x, t)
        xt = torch.cat([x_n, t_n], dim=1)
        return self.net(xt)

model = PINN(width=64, depth=4).to(device)

Cell 6 — PDE residual

In [ ]:
def pde_residual(model, x, t):
    x.requires_grad_(True)
    t.requires_grad_(True)

    c_hat = model(x, t)

    c_x = torch.autograd.grad(c_hat, x, torch.ones_like(c_hat), create_graph=True)[0]
    c_t = torch.autograd.grad(c_hat, t, torch.ones_like(c_hat), create_graph=True)[0]
    c_xx = torch.autograd.grad(c_x, x, torch.ones_like(c_x), create_graph=True)[0]

    return c_t + u*c_x - D*c_xx

Cell 7 — Sample training points

In [ ]:
rng = np.random.default_rng(0)

def sample_uniform(n, x0=0.0, x1=L, t0=0.0, t1=T_final):
    xs = rng.uniform(x0, x1, size=(n,1)).astype(np.float32)
    ts = rng.uniform(t0, t1, size=(n,1)).astype(np.float32)
    return xs, ts

def to_torch(a):
    return torch.from_numpy(a).to(device)

n_f = 4000
n_i = 1000
n_b = 1000
n_d = 2000

xf_np, tf_np = sample_uniform(n_f)

xi_np = rng.uniform(0.0, L, size=(n_i,1)).astype(np.float32)
ti_np = np.zeros_like(xi_np)
ci_np = xi_np.copy()  # c(x,0)=x

tb_np = rng.uniform(0.0, T_final, size=(n_b,1)).astype(np.float32)
xb0_np = np.zeros_like(tb_np)
xbL_np = np.full_like(tb_np, L)
cb0_np = np.zeros_like(tb_np)
cbL_np = np.full_like(tb_np, L)

xd_np, td_np = sample_uniform(n_d)
cd_np = c_groundtruth(xd_np, td_np)

xf, tf = to_torch(xf_np), to_torch(tf_np)
xi, ti, ci_t = to_torch(xi_np), to_torch(ti_np), to_torch(ci_np)
xb0, tb0, cb0_t = to_torch(xb0_np), to_torch(tb_np), to_torch(cb0_np)
xbL, tbL, cbL_t = to_torch(xbL_np), to_torch(tb_np), to_torch(cbL_np)
xd, td, cd_t = to_torch(xd_np), to_torch(td_np), to_torch(cd_np)


Cell 8 — Train (Adam)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

w_f, w_i, w_b, w_d = 1.0, 10.0, 10.0, 1.0

def train(epochs=3000, print_every=300):
    model.train()
    for ep in range(1, epochs+1):
        optimizer.zero_grad()

        r = pde_residual(model, xf.clone(), tf.clone())
        loss_f = torch.mean(r**2)

        loss_i = torch.mean((model(xi, ti) - ci_t)**2)

        loss_b = torch.mean((model(xb0, tb0) - cb0_t)**2) + torch.mean((model(xbL, tbL) - cbL_t)**2)

        loss_d = torch.mean((model(xd, td) - cd_t)**2)

        loss = w_f*loss_f + w_i*loss_i + w_b*loss_b + w_d*loss_d
        loss.backward()
        optimizer.step()

        if ep == 1 or ep % print_every == 0:
            print(f"ep={ep:5d}  total={loss.item():.3e}  pde={loss_f.item():.3e}  ic={loss_i.item():.3e}  bc={loss_b.item():.3e}  data={loss_d.item():.3e}")

train(epochs=3000, print_every=300)

Cell 9 — Accuracy vs ground truth

In [ ]:
model.eval()

Nx_test = 201
Nt_test = 101
x_test = np.linspace(0.0, L, Nx_test, dtype=np.float32).reshape(-1,1)
t_test = np.linspace(0.0, T_final, Nt_test, dtype=np.float32).reshape(-1,1)

XX, TT = np.meshgrid(x_test.reshape(-1), t_test.reshape(-1))
xq = XX.reshape(-1,1).astype(np.float32)
tq = TT.reshape(-1,1).astype(np.float32)

c_true = c_groundtruth(xq, tq)

with torch.no_grad():
    c_pred = model(to_torch(xq), to_torch(tq)).cpu().numpy()

mse = np.mean((c_pred - c_true)**2)
mae = np.mean(np.abs(c_pred - c_true))
rel_l2 = np.linalg.norm(c_pred - c_true) / np.linalg.norm(c_true)

print("MSE   =", mse)
print("MAE   =", mae)
print("RelL2 =", rel_l2)
